# Construcción de un chatBot

## Preprocesamiento de la información

Para preprocesar el texto voy a utilizar la libreria construida para NLP spaCy. La información que voya a tartar se recoge de una base de datos nutrida utilizando la API REST de Stack Overflow, que recoge preguntas y respuestas sobre Python y Machine learning.

Compruebo la conexión a la bd

In [ ]:
#!pip install 

In [86]:
import sqlite3
import spacy

nlp = spacy.load("en_core_web_sm")

In [111]:
con = sqlite3.connect('../../db/Stackoverflow.db')
cursor = con.cursor()
rows = cursor.execute("Select * from questions").fetchall()
questions = []
answers = []
tags =[]
vocabulary = []
vocabulary_lemma=[]
docs =[]
docs_lemma =[]
classes = []
for row in rows:
    id = row[0]
    doc = nlp(row[1])
    question = []
    question_lemma = []
    for token in doc:
        if token.is_alpha or token.is_digit:#limpio simbolos de puntuación, me quedo con caracteres y números
            question.append(token.text.lower())
            question_lemma.append(token.lemma_.lower())
    vocabulary.extend(question)
    vocabulary_lemma.extend(question_lemma)

    questions.append(row[1])
    answers.append(row[4])
    tagsRows = cursor.execute("Select t.tag from questions_tags as qt inner join tags as t on qt.id_tag=t.id where qt.id_question=?",(id,)).fetchall()
    tagList = []
    for tag in tagsRows:
        tagList.append(tag[0])
    classes.extend(tagList)
    tagList=sorted(tagList)
    tags.append(';'.join(tagList))
    docs.append((question,tagList))
    docs_lemma.append((question_lemma,tagList))
con.close()


In [112]:
vocabulary=list(set(sorted(vocabulary)))
vocabulary_lemma=list(set(sorted(vocabulary_lemma)))
print(len(vocabulary))
print(len(vocabulary_lemma))

4678
3968


In [114]:
classes = list(sorted(set(classes)))
tags =list(sorted(set(tags)))
print(len(tags))
print(len(classes))


3192
967


In [122]:
import numpy as np
def getBagWords(tokens,vocabulary):
    bag = []
    for word in vocabulary:
        if word in tokens:
            bag.append(1)
        else:
            bag.append(0)
    return np.array(bag)


bag = getBagWords(docs_lemma[7][0],vocabulary_lemma)
len(bag)

3968